In [37]:
import glob
import os
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import Point, box
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import Point, Polygon
from pyproj import Proj,Transformer
from rasterio.warp import calculate_default_transform, reproject, Resampling
import contextily as cx
from matplotlib_scalebar.scalebar import ScaleBar
import json
import shapely
import numpy as np
import changeos
from PIL import Image
from rasterio.windows import Window

def convert_coords(y, x, in_crs, out_crs):
    transformer = Transformer.from_crs(in_crs, out_crs)
    x, y = transformer.transform(x, y)

    return Point(x, y)

def move_pt(pt, x, y):
    return Point(pt.x+x, pt.y+y)

def find_tifs(pt, nov_tifs):
    tifs=[]
    for tif in nov_tifs:
        with rasterio.open(tif) as src:
            bbox_polygon = Polygon([(src.bounds.left, src.bounds.bottom),
                                    (src.bounds.right, src.bounds.bottom),
                                    (src.bounds.right, src.bounds.top),
                                    (src.bounds.left, src.bounds.top),
                                    (src.bounds.left, src.bounds.bottom)])
            if bbox_polygon.contains(pt):
                tifs.append(tif)
    return tifs

def reproj(src, target_crs):
    if type(src) == str:
        src = rasterio.open(src)

    source_crs = src.crs
    source_data = src.read()
    out_meta = src.meta.copy()

    transform, width, height = calculate_default_transform(
        source_crs, target_crs, src.width, src.height, *src.bounds
    )
    out_meta.update({'transform': transform, 'width': width, 'height': height, 'crs': target_crs})
    dst = rasterio.MemoryFile().open(**out_meta)

    reproject(
        source=source_data,
        destination=rasterio.band(dst, 1),
        src_transform=src.transform,
        src_crs=source_crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest,
    )

    return dst

def hist_match(source, reference):
    """
    Adjust the values of a source array
    so that its histogram matches that of a reference array
    """
    orig_shape = source.shape
    source = source.ravel()
    reference = reference.ravel()

    # get the set of unique pixel values
    # and their corresponding indices and counts
    s_values, s_idx, s_counts = np.unique(
        source, return_inverse=True, return_counts=True)
    r_values, r_counts = np.unique(reference, return_counts=True)

    # take the cumsum of the counts; empirical cumulative distribuition
    s_quantiles = np.cumsum(s_counts).astype(np.float64) / source.size
    r_quantiles = np.cumsum(r_counts).astype(np.float64) / reference.size

    # Create the lookup table,
    # find values in the reference corresponding to the quantiles in the source
    interp_r_values = np.interp(s_quantiles, r_quantiles, r_values)

    # using the inverted source indicies, map to the interpolated pixel values
    # and reshape to the original array
    return interp_r_values[s_idx].reshape(orig_shape)

def buffer_crop(src, point, buffer):
    buffer=box(point.x-buffer, point.y-buffer, point.x+buffer, point.y+buffer)    
    #read window of raster
    if type(src)==str:
        src=rasterio.open(src)
    gdf = gpd.GeoDataFrame({'geometry': buffer}, index=[0], crs=src.crs)

    out_image, out_transform = mask(src, gdf.geometry, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "bounds": buffer.bounds,
    })

    dataset=rasterio.MemoryFile().open(**out_meta)
    dataset.write(out_image)

    return dataset

def get_tifs(post_dir, pt):
    skysat_dir='../data/Gaza/skysat/'
    #folders=os.listdir(skysat_dir)
    post_tifs=[tif for tif in glob.glob(f'{skysat_dir}{post_dir}/*.tif')]
    pre_tifs=[tif for tif in glob.glob(f'{skysat_dir}pre_war/*.tif')]
    pre_tifs=find_tifs(pt, pre_tifs)
    post_tifs=find_tifs(pt, post_tifs)
    return pre_tifs, post_tifs

def generate_centroids(scale):
    admin=gpd.read_file('../data/Gaza/admin_bounds/GazaStrip_MunicipalBoundaries.shp')
    admin=admin.to_crs('epsg:32636')
    unosat=gpd.read_file('../data/Gaza/Gaza_20240503_2_footprints.csv')
    unosat['geometry']=unosat['.geo'].apply(lambda x: shapely.geometry.shape(json.loads(x)))
    unosat=gpd.GeoDataFrame(unosat, geometry='geometry', crs='EPSG:4326')
    unosat=unosat.to_crs('epsg:32636')
    #get bounds of unosat 
    xmin, ymin, xmax, ymax = unosat.total_bounds
    #grid of points 500m apart
    x=np.arange(xmin, xmax, (scale*2)-50)
    y=np.arange(ymin, ymax, (scale*2)-50)
    #x=np.arange(xmin, xmax, (scale*2))
    #y=np.arange(ymin, ymax, (scale*2))
    
    points=[]
    for i in x:
        for j in y:
            points.append(Point(i,j))
    points=gpd.GeoDataFrame(geometry=points, crs='epsg:32636')
    #keep points if they fall within admin 
    points=points[points.within(admin.geometry.union_all())]
    print(len(points))
    return points

def inference(pt,window,folder, unosat):
    model = changeos.from_name('changeos_r101')

    pre_tifs, post_tifs=get_tifs(folder, pt)
    try:
        pre=buffer_crop(pre_tifs[0], pt, window)
        post=buffer_crop(post_tifs[1], pt, window)
    except:
        try:
            post=buffer_crop(post_tifs[2], pt, window)
        except:
            return
    
    post_filename=post_tifs[0].split('/')[-1]
    post_filename=post_filename.split('.')[0]

    center=Point(pre.bounds[0]+window/2, pre.bounds[1]+window/2)
    x,y=int(center.x), int(center.y)

    outfile=f'../data/Gaza/skysat/{folder}_output/{post_filename}_{window}_{x}_{y}.csv'

    if os.path.exists(outfile):
        pre.close()
        post.close()
        return
    else:
        xmin, ymin, xmax, ymax = pre.bounds
        unosat=unosat.to_crs(post.crs)
        unosat=unosat.clip(box(xmin, ymin, xmax, ymax))
        centroids=unosat['geometry'].centroid
        coord_list=[(x.x, x.y) for x in centroids]

        pre_image=pre.read([1,2,3]).transpose(1,2,0)
        post_image=post.read([1,2,3]).transpose(1,2,0)
        pre_image=Image.fromarray(pre_image).resize((1024,1024))
        post_image=Image.fromarray(post_image).resize((1024,1024))

        loc, dam = model(pre_image, post_image)
        show(post.read([1,2,3]), transform=post.transform)

        post.write(dam, 4)
        
        unosat['damage']=[x[3] for x in post.sample(coord_list)]
        unosat.to_csv(outfile, index=False)
        print(post)
        show(post.read([1,2,3]), transform=post.transform)
        plt.show()
        with rasterio.open(outfile.replace('.csv','.tif'), 'w', **post.meta) as dst:
            dst.write(post.read([]))
        pre.close()
        post.close()
        return

In [38]:
import pandas as pd 
from pandarallel import pandarallel
from joblib import Parallel, delayed
import tqdm 
from tqdm.auto import tqdm

model = changeos.from_name('changeos_r101')

def parallel_inference(scale, folder):
    #pandarallel.initialize(progress_bar=True, nb_workers=4)
    unosat=gpd.read_file('../data/Gaza/Gaza_20240503_2_footprints.csv')
    unosat['geometry']=unosat['.geo'].apply(lambda x: shapely.geometry.shape(json.loads(x)))
    unosat=gpd.GeoDataFrame(unosat, geometry='geometry', crs='EPSG:4326')
    unosat=unosat[['geometry', 'class', 'FID']]
    points=generate_centroids(scale)
    out_folder=f'../data/Gaza/skysat/{folder}_output'
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    #points.parallel_apply(lambda x: inference(x['geometry'], scale, folder, unosat), axis=1)
    #Parallel(n_jobs=3)(delayed(inference)(x, scale, folder, unosat) for x in tqdm(points['geometry']))
    inference(points['geometry'].iloc[100], scale, folder, unosat)


parallel_inference(400, '2024_05')
#parallel_inference(400, '2024_01')
#parallel_inference(400, '2023_11')



639


In [1]:
import numpy as np
import changeos
from PIL import Image

def plot_row(pt,window, folder):
    model = changeos.from_name('changeos_r101') # take 'changeos_r101' as example

    pre_tifs, post_tifs=get_tifs(folder, pt)
    
    #unosat=gpd.read_file('../data/Gaza/UNOSAT/UNOSAT_GAZA_20240503.shp')
    unosat=gpd.read_file('../data/Gaza/Gaza_20240503_2_footprints.csv')
    unosat['geometry']=unosat['.geo'].apply(lambda x: shapely.geometry.shape(json.loads(x)))
    unosat=gpd.GeoDataFrame(unosat, geometry='geometry', crs='EPSG:4326')

    fig, ax = plt.subplots(1,4, figsize=(14, 4))

    
    pre=buffer_crop(pre_tifs[0], pt, window)
    post=buffer_crop(post_tifs[0], pt, window)

    xmin, ymin, xmax, ymax = pre.bounds
    unosat=unosat.to_crs(post.crs)
    unosat=unosat.clip(box(xmin, ymin, xmax, ymax))
    damaged=unosat[unosat['class']=='1']
    undamaged=unosat[unosat['class']=='0']

    pre_rgb=hist_match(pre.read([1,2,3]), post.read([1,2,3])).astype(np.uint8)
    show(pre_rgb,transform=pre.transform, ax=ax[0])
    #unosat.plot(ax=ax[0], color='red', alpha=0.6,markersize=10)

    show(post.read([1,2,3]),transform=post.transform, ax=ax[1])
    #unosat.plot(ax=ax[1], color='red', alpha=0.6,markersize=10)
    #create pil image

    pre_image=pre.read([1,2,3]).transpose(1,2,0)
    post_image=post.read([1,2,3]).transpose(1,2,0)
    #rescale the images to 1024x1024 
    pre_image=Image.fromarray(pre_image).resize((1024,1024))
    post_image=Image.fromarray(post_image).resize((1024,1024))


    #print(pre_image.shape)
    #print(post_image.shape)
    loc, dam = model(pre_image, post_image)
    loc, dam = changeos.visualize(loc, dam)
    # # plot the prediction
    unosat.plot(ax=ax[2], color='red', alpha=0.6,markersize=10)
    ax[3].imshow(dam)

    titles=['Pre Destruction', 'Post Destruction', 'UNOSAT', 'ChangeOS']
    i=0
    for a in ax:
        scalebar = ScaleBar(1, units='m', location='lower right')

        #a.set_xlim(xmin, xmax)
        #a.set_ylim(ymin, ymax)
        a.axis('off')
        # remove axis ticklabels 
        #a.set_xticklabels([])
        #a.set_yticklabels([])
        #increase fontsize for title 
        a.set_title(titles.pop(0), fontsize=20)
        a.add_artist(scalebar)

    plt.tight_layout()
    #plt.savefig('../figs/pwtt_coh_comparison2.png', dpi=300)
    plt.show()


beit_hanoun=convert_coords(34.522359,31.546570, 'epsg:4326','epsg:32636')
central_gaza=convert_coords(34.40163,31.43744, 'epsg:4326','epsg:32636')
nw_gaza=Point(633289.384,3482754.016)
low_density=Point(633359.34,3481677.71)

pt=beit_hanoun
#window=256
window=100
pt=move_pt(pt, 100, 200)
#350 seems best
windows=range(200, 500, 20)
#for window in windows:
#     print(window)
     #plot_row(low_density, window)
plot_row(pt, 300, '2023_11')
#plot_row(low_density, window)

NameError: name 'convert_coords' is not defined

In [26]:
pt=points.iloc[1400].geometry
#plot_row(pt, 400)
windows=range(300, 500, 20)
for window in windows:
     print(window)
     plot_row(pt, window)


NameError: name 'points' is not defined

In [20]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd 
import glob
import geopandas as gpd
import shapely
import json

scale='400'
folder='2024_05'
unosat=gpd.read_file('../data/Gaza/Gaza_20240107_2_footprints.csv')

unosat['geometry']=unosat['.geo'].apply(lambda x: shapely.geometry.shape(json.loads(x)))
unosat=gpd.GeoDataFrame(unosat, geometry='geometry', crs='EPSG:4326')
fp=pd.DataFrame()
for file in glob.glob(f'../data/Gaza/skysat/{folder}_output/*.csv'):
    filename=file.split('/')[-1]
    file_scale=filename.split('_')[-3]
    if file_scale==scale:
        df=pd.read_csv(file)
        fp=pd.concat([fp, df])

fp['FID']=fp['FID'].astype(int)
unosat['FID']=unosat['FID'].astype(int)
fp = fp.drop_duplicates(subset='FID')
merged=unosat.merge(fp[['FID','damage']], on='FID', how='left')
merged['pred']=np.where(merged['damage']>0, 1, 0)
merged['class']=merged['class'].astype(int)
merged['area']=merged['area'].astype(float)
print('Missing: ', merged['damage'].isna().sum())

Missing:  22457


In [22]:
from sklearn.metrics import f1_score, precision_score, recall_score
merged=merged.dropna(subset='damage')
f1 = f1_score(merged['class'], merged['pred'], sample_weight=merged['area'], average='weighted')
precision = precision_score(merged['class'], merged['pred'], sample_weight=merged['area'], average='weighted')
recall = recall_score(merged['class'], merged['pred'], sample_weight=merged['area'], average='weighted')

print(f1, precision, recall)

0.6801441580402602 0.6861945620889401 0.6759510520705297
